In [ ]:
# En el archivo: notebooks/02_prueba_buscador_faq.ipynb

# --- CELDA 1: Importar las funciones ---
# Asegúrate de que el notebook se ejecuta desde la carpeta raíz del proyecto
import sys
import os

# Añadimos la carpeta 'src' al path para que Python la encuentre
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))

from tools.buscador import buscar_en_faq, buscador_de_reglamentos

# --- CELDA 2: Probar el buscador de FAQ ---
print("--- Probando Buscador de FAQ ---")

pregunta_faq_1 = "¿dónde encuentro el correo de soporte?"
print(f"Pregunta: {pregunta_faq_1}")
print(f"Respuesta: {buscar_en_faq(pregunta_faq_1)}\n")

pregunta_faq_2 = "¿cómo hago para cancelar una materia?"
print(f"Pregunta: {pregunta_faq_2}")
print(f"Respuesta: {buscar_en_faq(pregunta_faq_2)}\n")

pregunta_faq_3 = "¿a qué hora abre la biblioteca?"
print(f"Pregunta: {pregunta_faq_3}")
print(f"Respuesta: {buscar_en_faq(pregunta_faq_3)}\n")


# --- CELDA 3: Probar el buscador de Reglamentos ---
# La primera vez que se ejecute esta celda, puede tardar un poco mientras
# descarga el modelo de embeddings 'all-MiniLM-L6-v2'.
print("--- Probando Buscador de Reglamentos ---")

pregunta_reglamento_1 = "¿cuál es la nota mínima para aprobar?"
print(f"Pregunta: {pregunta_reglamento_1}")
print(f"Respuesta: {buscador_de_reglamentos.buscar(pregunta_reglamento_1)}\n")

pregunta_reglamento_2 = "¿qué porcentaje de asistencia necesito?"
print(f"Pregunta: {pregunta_reglamento_2}")
print(f"Respuesta: {buscador_de_reglamentos.buscar(pregunta_reglamento_2)}\n")

pregunta_reglamento_3 = "¿cuántas oportunidades tengo para el examen final?"
print(f"Pregunta: {pregunta_reglamento_3}")
print(f"Respuesta: {buscador_de_reglamentos.buscar(pregunta_reglamento_3)}\n")